In [ ]:
def get_census_data(zip_code):
    # Your Census API key
    api_key = '72464ba87c05672495fbe0ce0b4f7076c109c8be'
    
    # Base URL for Census API
    base_url = 'https://api.census.gov/data'

    # Year of the data
    year = '2020'

    # Dataset (e.g., acs/acs5 for the American Community Survey 5-Year Data)
    dataset = 'acs/acs5'

    # Variables to retrieve
    # B01003_001E is the variable for total population
    # B19013_001E is the variable for median household income
    variables = 'B01003_001E,B19013_001E'

    # Building the complete API URL
    url = f'{base_url}/{year}/{dataset}?get={variables}&for=zip%20code%20tabulation%20area:{zip_code}&key={api_key}'

    # Make the API request
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()
        # The first element is headers, and the second is the values
        headers, values = data[0], data[1]
        return dict(zip(headers, values))
    else:
        return f'Error: {response.status_code}'

# Example usage
zip_code = '10463'
census_data = get_census_data(zip_code)
print(census_data)


In [ ]:
def write_to_csv_with_census_data(filename):
    """
    Appends census data to each row in the CSV file and writes it back to the file.

    :param filename: Name of the CSV file to read from and write to.
    """
    # Read the existing CSV file
    df = pd.read_csv(filename)

    # Iterate over each row and get census data
    for index, row in df.iterrows():
        census_data = get_census_data(str(row['ZIP Code']))        
        # Check if the returned value is a dictionary
        if isinstance(census_data, dict):
            # Rename the keys and add to DataFrame
            df.at[index, 'Population'] = census_data['B01003_001E']
            df.at[index, 'Average Household Income'] = census_data['B19013_001E']
        else:
            # Handle the case where census_data is not a dictionary (e.g., an error message)
            df.at[index, 'Population'] = 0
            df.at[index, 'Average Household Income'] = 0

    # Write updated DataFrame to CSV
    df.to_csv(filename, index=False)

# Example usage:
csv_filename = 'nyc_zip_mapping.csv'
write_to_csv_with_census_data(csv_filename)


In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import HeatMap

# Load New York city neighborhood boundaries
nyc_neighborhoods = gpd.read_file('d085e2f8d0b54d4590b1e7d1f35594c1pediacitiesnycneighborhoods.json')

# Convert to a projected CRS (EPSG:2263 is commonly used for New York)
nyc_neighborhoods_projected = nyc_neighborhoods.to_crs(epsg=2263)

# Now calculate centroids in the projected CRS
nyc_neighborhoods_projected['centroid'] = nyc_neighborhoods_projected.geometry.centroid

# Convert centroids back to the geographic CRS (WGS84, EPSG:4326) for mapping
nyc_neighborhoods['centroid'] = nyc_neighborhoods_projected['centroid'].to_crs(epsg=4326)

# Extract latitude and longitude
nyc_neighborhoods['lat'] = nyc_neighborhoods['centroid'].y
nyc_neighborhoods['lon'] = nyc_neighborhoods['centroid'].x

# Create a map centered around New York City
m = folium.Map(location=[40.7128, -74.0060], zoom_start=11)

# Add the heatmap
heat_data = [[row['lat'], row['lon']] for index, row in nyc_neighborhoods.iterrows()]
HeatMap(heat_data).add_to(m)

# Save the map to an HTML file
m.save('nyc_population_heatmap.html')

# Display the map in Jupyter Notebook (if you're using one)
m

In [ ]:
def get_zip_info(zip_code,csv_file="nyc_zip_mapping.csv"):
    # Read the CSV file
    df = pd.read_csv(csv_file)
    if type(zip_code) == str:
        zip_code = int(zip_code)

    # Find the row corresponding to the ZIP code
    zip_info = df[df['ZIP Code'] == zip_code]

    if not zip_info.empty:
        return zip_info.to_dict('records')[0]
    else:
        return "ZIP code not found."

